In [2]:
import env
import os
import acquire as a
import prepare as p
import wrangle as w
#import modeling as m
#import evaluate as e
# Ignore Warning
import warnings
warnings.filterwarnings("ignore")
# Array and Dataframes
import numpy as np
import pandas as pd
# Evaluation: Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import MinMaxScaler

In [3]:
zillow = a.zillow()

File exists, pulling from system.


In [4]:
zillow.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,10712236,1.0,NaN,NaN,2.0,3.0,NaN,8.0,2.0,NaN,...,6.037113e+13,0.153774,2017-01-06,Central,NaN,NaN,Central,NaN,NaN,NaN
1,10712908,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,NaN,...,6.037113e+13,-0.132414,2017-01-12,Central,NaN,NaN,Central,NaN,NaN,NaN
2,10712949,1.0,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,...,6.037113e+13,0.003707,2017-01-13,Central,NaN,NaN,Central,NaN,NaN,NaN
3,10713127,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,NaN,...,6.037113e+13,0.013564,2017-01-13,NaN,NaN,NaN,Central,NaN,NaN,NaN
4,10716435,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,NaN,...,6.037134e+13,0.029961,2017-01-17,NaN,NaN,NaN,Central,NaN,NaN,NaN


In [5]:
train, validate, test = a.splitter(zillow)

Dataframe: (2140882, 67) 100%
Train: (1370164, 67) | ~60%
Validate: (342541, 67) | ~20%
Test: (428177, 67) | ~20%


In [6]:
w.summarize(train)

=====   SHAPE   =====
(1370164, 67)
=====   INFO   =====
<class 'pandas.core.frame.DataFrame'>
Index: 1370164 entries, 1575798 to 1233670
Data columns (total 67 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   parcelid                      1370164 non-null  int64  
 1   airconditioningtypeid         329130 non-null   float64
 2   architecturalstyletypeid      1537 non-null     float64
 3   basementsqft                  991 non-null      float64
 4   bathroomcnt                   1370164 non-null  float64
 5   bedroomcnt                    1370164 non-null  float64
 6   buildingclasstypeid           0 non-null        float64
 7   buildingqualitytypeid         908452 non-null   float64
 8   calculatedbathnbr             1369182 non-null  float64
 9   decktypeid                    7930 non-null     float64
 10  finishedfloor1squarefeet      104608 non-null   float64
 11  calculatedfinishedsquarefeet  1

In [24]:
def handle_missing_values(df, column_percent=.6, row_percent=.6):
    '''
    Values in 'column_percent' & 'row_percent' should be a decimal between 0-1
    this will indicate how much of the values or columns will be retained based on percentage of missing values.

    The higher the decimal the lower the threshold, vice versa. It will be the inverse of what is put -- (ex. 0.8 means values that have at least 80%
    of none nulls will not be dropped.)
    '''

    col_limit = int(len(df.columns) * column_percent)
    row_limit = int(len(df.columns) * row_percent)
    df.dropna(thresh=col_limit,axis=1,inplace=True)
    df.dropna(thresh=row_limit,axis=0,inplace=True)

    return df

In [25]:
zillowcopy = zillow.copy()

In [26]:
df = handle_missing_values(zillowcopy)
df

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,heatingorsystemdesc,storydesc,typeconstructiondesc
1636,14081891,NaN,7.0,NaN,2.5,3.0,NaN,2.5,NaN,NaN,...,NaN,NaN,6.059100e+13,0.011913,2017-01-03,NaN,Contemporary,Forced air,NaN,Frame
1705,14159038,1.0,7.0,NaN,2.0,3.0,NaN,2.0,NaN,NaN,...,NaN,NaN,6.059002e+13,0.030711,2017-01-12,Central,Contemporary,Forced air,NaN,Frame
1725,14172985,1.0,7.0,NaN,2.5,4.0,NaN,2.5,NaN,NaN,...,NaN,NaN,6.059012e+13,0.007491,2017-01-03,Central,Contemporary,Forced air,NaN,Frame
1775,14215602,1.0,NaN,NaN,2.0,3.0,NaN,2.0,NaN,NaN,...,NaN,NaN,6.059022e+13,0.019404,2017-01-05,Central,NaN,Forced air,NaN,Frame
1828,14287849,1.0,7.0,NaN,2.0,3.0,NaN,2.0,NaN,NaN,...,NaN,NaN,6.059053e+13,-0.026650,2017-01-09,Central,Contemporary,Forced air,NaN,Frame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136618,17097816,NaN,NaN,273.0,2.5,3.0,NaN,2.5,66.0,1656.0,...,NaN,NaN,6.111001e+13,0.018000,2017-03-08,NaN,NaN,NaN,Basement,NaN
2138472,17194590,NaN,NaN,NaN,5.5,5.0,NaN,5.5,66.0,3253.0,...,NaN,NaN,6.111006e+13,-0.023768,2017-05-03,NaN,NaN,NaN,NaN,NaN
2138750,17207797,NaN,NaN,670.0,2.5,3.0,NaN,2.5,66.0,2520.0,...,NaN,NaN,6.111005e+13,NaN,NaN,NaN,NaN,NaN,Basement,NaN
2140509,17284977,NaN,NaN,209.0,6.5,6.0,NaN,6.5,66.0,7530.0,...,NaN,NaN,6.111007e+13,NaN,NaN,NaN,NaN,NaN,Basement,NaN


# Mall Customers

1. Acquire data from the customers table in the mall_customers database.

In [10]:
mall = a.mall()

File exists, pulling from system.


In [11]:
w.summarize(mall)

=====   SHAPE   =====
(200, 5)
=====   INFO   =====
<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 9.4+ KB
None
=====   DESCRIBE   =====
|                |   count |   mean |     std |   min |   25% |   50% |    75% |   max |
|:---------------|--------:|-------:|--------:|------:|------:|------:|-------:|------:|
| customer_id    |     200 | 100.5  | 57.8792 |     1 | 50.75 | 100.5 | 150.25 |   200 |
| age            |     200 |  38.85 | 13.969  |    18 | 28.75 |  36   |  49    |    70 |
| annual_income  |     200 |  60.56 | 26.2647 |    15 | 41.5  |  61.5 |  78    |   137 |
| spending_score

In [12]:
age_q1, age_q3 = mall['age'].quantile([.25,.75])

In [13]:
age_iqr = age_q3 - age_q1

In [14]:
age_upper = age_q3 * 1.5
age_lower = age_q1 * 1.5

In [15]:
mall[mall['age'] > age_upper]

,customer_id,gender,age,annual_income,spending_score


In [16]:
train, validate, test = a.splitter(mall)

Dataframe: (200, 5) 100%
Train: (128, 5) | ~60%
Validate: (32, 5) | ~20%
Test: (40, 5) | ~20%


In [17]:
train = pd.get_dummies(train, drop_first=True)
train

,customer_id,age,annual_income,spending_score,gender_Male
129,130,38,71,75,True
190,191,34,103,23,False
35,36,21,33,81,False
130,131,47,71,9,True
147,148,32,77,74,False
...,...,...,...,...,...
157,158,30,78,78,False
18,19,52,23,29,True
183,184,29,98,88,False
52,53,31,43,54,False
